In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

In [5]:
# Define the training data
text = "Hello, how are you today? I am doing well, thank you!"

# Create a vocabulary
chars_list= list(set(text))

vocab_size = len(chars_list)
char_to_index = {char: index for index, char in enumerate(chars_list)}
index_to_char = {index: char for index, char in enumerate(chars_list)}

# Convert text to a sequence of indices
text_as_indices = [char_to_index[char] for char in text]

print(text_as_indices)

# Define the LSTM-based Language Model
class CharLM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, *args, **kwargs):
        
        super().__init__(*args, **kwargs)
        
        super(CharLM, self).__init()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x, hidden):
        out = self.embedding(x)
        out, hidden = self.lstm(out, hidden)
        out = self.fc(out)
        return out, hidden

# Hyperparameters
input_size = vocab_size
hidden_size = 128
num_layers = 2
output_size = vocab_size
learning_rate = 0.01
num_epochs = 100

# Create the model
model = CharLM(input_size, hidden_size, num_layers, output_size)

# Loss and optimizer

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training the model
for epoch in range(num_epochs):
    inputs = torch.tensor(text_as_indices[:-1]).unsqueeze(0)
    targets = torch.tensor(text_as_indices[1:]).unsqueeze(0)

    hidden = (torch.zeros(num_layers, 1, hidden_size),
              torch.zeros(num_layers, 1, hidden_size))

    outputs, _ = model(inputs, hidden)
    loss = criterion(outputs.view(-1, vocab_size), targets.view(-1))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

# Generating text
with torch.no_grad():
    seed = "Hello"
    seed_indices = [char_to_index[char] for char in seed]
    generated_text = seed

    hidden = (torch.zeros(num_layers, 1, hidden_size),
              torch.zeros(num_layers, 1, hidden_size))

    for _ in range(100):
        input_char = torch.tensor(seed_indices[-1]).unsqueeze(0).unsqueeze(0)
        output, hidden = model(input_char, hidden)
        _, predicted_index = torch.max(output, 2)
        next_char = index_to_char[predicted_index.item()]
        generated_text += next_char
        seed_indices.append(predicted_index.item())

    print(generated_text)


[21, 18, 8, 8, 14, 9, 19, 1, 14, 3, 19, 16, 2, 18, 19, 7, 14, 0, 19, 5, 14, 17, 16, 7, 12, 19, 10, 19, 16, 13, 19, 17, 14, 11, 4, 20, 19, 3, 18, 8, 8, 9, 19, 5, 1, 16, 4, 6, 19, 7, 14, 0, 15]


AttributeError: 'super' object has no attribute '_CharLM__init'